# Explore here

In [ ]:
import requests
import pandas as pd

url= "https://api.worldbank.org/v2/indicator"
params = {
    "format" : "json",
    "per_page": 500,
    "page": 1
}

response = requests.get(url, params=params, timeout=30)
response.raise_for_status() #similar to try catch error
payload = response.json()
print(response.status_code)
print(payload[0]['pages'])
total_pages = payload[0]['pages']
all_indicators_list = []
for page in range(1, total_pages + 1):
    params["page"] = page
    response = requests.get(url, params=params, timeout=30)
    response.raise_for_status()
    payload = response.json()
    for indicator in payload[1]:
        all_indicators_list.append({"id": indicator['id'], "name": indicator['name']})
  
    



200
59
total indicators: 29323


The goal is to analyze the socioeconomic and environmental evolution of 5 countries chosen by you between 2010 and 2024.

Countries: Spain, Mexico, Venezuela, Argentina, Colombia
Indicators: GDP, Trade, Exports, GINI, Tobacco, Alcohol

In [33]:
df_indicators = pd.DataFrame(all_indicators_list)
# print(df_indicators.head())
selected_ids = [
    "SH.PRV.SMOK",      # tobacco
    "SH.ALC.PCAP.LIT",  # alcohol
    "NY.GDP.PCAP.CD",   # GDP per capita
    "NE.EXP.GNFS.CD",   # exports
    "SI.POV.GINI",      # Gini index
    "SP.POP.TOTL",      # population
    "SP.DYN.LE00.IN",   # life expectancy
    "SP.POP.GROW",      # population growth
]
selected_indicators = {}
for ind_id in selected_ids:
    if ind_id in df_indicators['id'].values:
        row = df_indicators[df_indicators["id"] == ind_id].iloc[0]  # get the row
        selected_indicators[ind_id] = {
            "id": row["id"],
            "name": row["name"]
        }
print(selected_indicators)

{'SH.PRV.SMOK': {'id': 'SH.PRV.SMOK', 'name': 'Prevalence of current tobacco use (% of adults)'}, 'NY.GDP.PCAP.CD': {'id': 'NY.GDP.PCAP.CD', 'name': 'GDP per capita (current US$)'}, 'NE.EXP.GNFS.CD': {'id': 'NE.EXP.GNFS.CD', 'name': 'Exports of goods and services (current US$)'}, 'SI.POV.GINI': {'id': 'SI.POV.GINI', 'name': 'Gini index'}, 'SP.POP.TOTL': {'id': 'SP.POP.TOTL', 'name': 'Population, total'}, 'SP.DYN.LE00.IN': {'id': 'SP.DYN.LE00.IN', 'name': 'Life expectancy at birth, total (years)'}, 'SP.POP.GROW': {'id': 'SP.POP.GROW', 'name': 'Population growth (annual %)'}}
